In [1]:
import os
import numpy as np
from textblob import TextBlob

from spacy.en import English
nlp=English()
import inflect
p= inflect.engine()

data_dir='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1'

train_cp_dir=data_dir+'/'+'Train_catches'
train_docs_dir=data_dir+'/'+'Train_docs'


import stop_words
stop_words=stop_words.get_stop_words('en')
stop_words_2=['i','me','we','us','you','u','she','her','his','he','him','it','they','them','who','which','whom','whose','that','this','these','those','anyone','someone','some','all','most','himself','herself','myself','itself','hers','ours','yours','theirs','to','in','at','for','from','etc',' ',',']
stop_words.extend(stop_words_2)
stop_words.extend(['with', 'at', 'from', 'into', 'during', 'including', 'until', 'against', 'among', 'throughout', 'despite', 'towards', 'upon', 'concerning', 'of', 'to', 'in', 'for', 'on', 'by', 'about', 'like', 'through', 'over', 'before', 'between', 'after', 'since', 'without', 'under', 'within', 'along', 'following', 'across', 'behind', 'beyond', 'plus', 'except', 'but', 'up', 'out', 'around', 'down', 'off', 'above', 'near', 'and', 'or', 'but', 'nor', 'so', 'for', 'yet', 'after', 'although', 'as', 'as', 'if', 'long', 'because', 'before', 'even', 'if', 'even though', 'once', 'since', 'so', 'that', 'though', 'till', 'unless', 'until', 'what', 'when', 'whenever', 'wherever', 'whether', 'while', 'the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'yours', 'his', 'her', 'its', 'ours', 'their', 'few', 'many', 'little', 'much', 'many', 'lot', 'most', 'some', 'any', 'enough', 'all', 'both', 'half', 'either', 'neither', 'each', 'every', 'other', 'another', 'such', 'what', 'rather', 'quite'])
stop_words=list(set(stop_words))

common_phrase_dict={}
dist=[]
L_dict={}


import nltk
# grammar = ('''
#     NP: {<DET>?<ADJ>*<NOUN>+} # NP
#     PNP: {<NP><ADP><NP>} #PP
#     K:{<NP>|<ADJ>}
#     PNP: {<K><ADP><PNP>+} #PP
    
#     ''')

grammar = ('''
    NP: {<DET>?<ADJ>*<NOUN>+} # NP
    PREP:{<ADP>|<PRT>} # prep
    PNP: {<NP><PREP><NP>} #PP
    K:{<NP>|<ADJ>}
    PNP2: {<K><PREP><PNP>+} #PP
    VB:{<VERB>|<ADV>}
    VNP:{<VB><NP>} # VNP
    
    ''')

chunkParser = nltk.RegexpParser(grammar)

def leaves(tree,elems):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() in elems):
        yield subtree.leaves()

def get_terms(tree,elems):
    for leaf in leaves(tree,elems):
        yield [w for w,t in leaf]



/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  return f(*args, **kwds)


In [2]:
def refine_noun_phrase(np_text):
    ref_np=[]
    try:
        dummy_np_text= str(np_text)
        words= nlp(dummy_np_text)

        if words[0].pos_=='DET' or words[0].tag_=='PRP$':
            dummy_np_text= ' '.join([words[i].text for i in range(1,len(words))])
        
        
        ref_np.append(dummy_np_text)
        words=nlp(dummy_np_text)
        
        if len(words)==0:
            return ref_np

        if words[0].pos_=='ADJ' and len(words)>=2:
            dummy_np_text= ' '.join([words[i].text for i in range(1,len(words))])
            ref_np.append(dummy_np_text)

        singulars=[]
        for elem in ref_np:
            if p.singular_noun(elem)!=False:
                singulars.append(p.singular_noun(elem))

        ref_np.extend(singulars)
    except Exception as e:
        print('Exception e')
        print(np_text, dummy_np_text)
    
    return ref_np
        

In [3]:

for i in range(0,100):
    doc_file='case_'+str(i)+'_statement.txt'
    cp_file='case_'+str(i)+'_catchwords.txt'
    doc_file=open(train_docs_dir+'/'+doc_file,errors='ignore')
    cp_file=open(train_cp_dir+'/'+cp_file,errors='ignore')


    for line in cp_file:
        phrase_list=line.strip().lower().split(',')
    phrase_list=[i.strip() for i in phrase_list]
    L=len(phrase_list)
    
    if L not in L_dict:
        L_dict[L]=0
    L_dict[L]+=1
    
    text=''
    for line in doc_file:
        text=text+line+' '

    doc=nlp(text)
    noun_phrases=[]

    for np in doc.noun_chunks:
        np_text=np.text.lower()
        noun_phrases.append(np_text)

    for word in doc:
        if word.pos_ =='NOUN' or word.pos_=='PROPN' or word.pos_=='ADJ':
            noun_phrases.append(word.text.lower())
    
    
    tagged = nltk.pos_tag(nltk.word_tokenize(text.lower()),tagset='universal')    
    tree = chunkParser.parse(tagged)
    
    terms=get_terms(tree,['NP','PNP','PNP2','VNP'])
    for term in terms:
        w=''
        for word in term:
            w+=word+' '
            
        noun_phrases.append(w.strip())
        
    a=[]
    for np in noun_phrases:
        a.extend(refine_noun_phrase(np.lower()))
    
    noun_phrases=list(a)
    
    left= len(set(phrase_list)- set(noun_phrases))/ len(set(phrase_list))
    print(L,'\t',left)
       
    if left > 0.3 and L>10:
        print(left)
        print(cp_file)
        print(phrase_list)
        print()
        print(set(phrase_list)- set(noun_phrases))     
        print()
        print(set(noun_phrases))
        print('\n')
    
    text=text.lower()
    c=0
    for phrase in phrase_list:
        if phrase in text:
            c+=1

    common_phrase_dict[i]=((L,c,c/L))
    dist.append(c/L)

print(L_dict)

import numpy as np
print('Min', min(dist))
print('Max', max(dist))
print('Mean', np.mean(dist))
print('Median', np.median(dist))

111 	 0.18018018018018017
1 	 0.0
9 	 0.2222222222222222
12 	 0.0
8 	 0.0
12 	 0.0
28 	 0.10714285714285714
2 	 1.0
87 	 0.11494252873563218
77 	 0.16883116883116883
1 	 0.0
67 	 0.26865671641791045
14 	 0.0
3 	 0.0
6 	 0.16666666666666666
4 	 0.0
11 	 0.09090909090909091
164 	 0.1402439024390244
15 	 0.26666666666666666
22 	 0.045454545454545456
2 	 0.0
7 	 0.2857142857142857
26 	 0.07692307692307693
1 	 0.0
31 	 0.12903225806451613
13 	 0.23076923076923078
33 	 0.21212121212121213
1 	 0.0
4 	 0.25
54 	 0.25925925925925924
6 	 0.16666666666666666
31 	 0.4838709677419355
0.4838709677419355
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_31_catchwords.txt' mode='r' encoding='UTF-8'>
['affirmation', 'capital offence', 'causing disappearance', 'commission of the offence', 'commit an offence', 'conspiracy', 'contrary to law', 'corruptly making report', 'criminal conspiracy', 'culpability', 'delinquent', 'disappearance of evidence', 'false information

76 	 0.15789473684210525
5 	 0.2
42 	 0.11904761904761904
12 	 0.4166666666666667
0.4166666666666667
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_35_catchwords.txt' mode='r' encoding='UTF-8'>
['attesting witness', 'consideration', 'investigation', 'notice', 'partition', 'pending', 'possession', 'property', 'right', 'sale', 'sale deed', 'transferred']

{'notice', 'pending', 'transferred', 'attesting witness', 'right'}

{'', 'one lakshmiammal', 'unsuccessful party before the trial court', 'vide', 'applicant', 'k.r', 'uddin', 'hearing', 'merit', 'himself', 'or', 'approach of the high court', 'jurisdiction', 'india', 'andal', 'provision', 'mohan', 'i.a.', 'obviou', 'das', 'intestate leaving legal heirs', 'evidence at an earlier stage', 'three condition', 'inc.', 'council in kessowji issur v. g.i.p', 'plaintiff', 'i', 'justice', 'subburaj', 'that interim stage', 'legal heirs', 'learned', 'i.e', 'pronouncement', 'production of additional evidence', 

46 	 0.1956521739130435
64 	 0.296875
27 	 0.14814814814814814
13 	 0.23076923076923078
1 	 0.0
12 	 0.08333333333333333
8 	 0.25
14 	 0.07142857142857142
9 	 0.1111111111111111
23 	 0.17391304347826086
30 	 0.16666666666666666
27 	 0.37037037037037035
0.37037037037037035
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_47_catchwords.txt' mode='r' encoding='UTF-8'>
['adjudication', 'application of', 'application under', 'approval under', 'compensation', 'complaint under', 'conditions of service', 'contravention', 'discharge', 'discharge or dismissal', 'dismissal', 'dismissal of workman', 'industrial dispute', 'industrial tribunal', 'interpretation of statute', 'misconduct', 'object of section', 'payment of wage', 'pendency of industrial dispute', 'pendency of proceeding', 'reference under', 'reinstatement', 'relationship of employer and employee', 'remedy under', 'social justice', 'standing order', 'unfair labour practice']

{'pendency of proceedi

43 	 0.046511627906976744
27 	 0.1111111111111111
22 	 0.045454545454545456
4 	 0.0
5 	 0.0
10 	 0.4
47 	 0.10638297872340426
43 	 0.34782608695652173
0.34782608695652173
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_55_catchwords.txt' mode='r' encoding='UTF-8'>
['administrative action', 'alternative remedy', 'bonded labour', 'commercial transaction', 'earnest money', 'enforceable right', 'exemption of sales tax', 'interim order', 'irregularities', 'judicial review', 'legitimate expectation', 'locus standi', 'locus standi to file', 'malice', 'natural justice', 'notice under', 'power of judicial review', 'public auction', 'public interest litigation', 'rule of law', 'secretary of state', 'without prejudiceadministrative action', 'alternative remedy', 'bonded labour', 'commercial transaction', 'earnest money', 'enforceable right', 'exemption of sales tax', 'interim order', 'irregularities', 'judicial review', 'legitimate expectation', 'locus stan

85 	 0.2
112 	 0.125
39 	 0.047619047619047616
4 	 0.25
10 	 0.6
46 	 0.32608695652173914
0.32608695652173914
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_61_catchwords.txt' mode='r' encoding='UTF-8'>
['accident', 'apportionment', 'appropriate multiplier', 'award', 'challenge', 'claim', 'compensation', 'compensation amount', 'compensation for', 'compensation for loss', 'compensation payable', 'consortium', 'damage', 'date of', 'death', 'dependency', 'disability', 'earning', 'enhancement of compensation', 'estate', 'father', 'funeral expenses', 'grant', 'hospital', 'information', 'insurance company', 'interest', 'just and reasonable', 'loss of consortium', 'loss of dependency', 'loss of estate', 'loss of love and affection', 'love and affection', 'medical expenses', 'medical treatment', 'motor accident', 'multiplier', 'of deceased', 'pass', 'permanent disability', 'personal expenses', 'registration', 'review', 'transport', 'treatment', 'welfare

20 	 0.05
3 	 0.0
7 	 0.14285714285714285
4 	 0.0
62 	 0.27419354838709675
4 	 0.0
22 	 0.045454545454545456
14 	 0.07142857142857142
81 	 0.1728395061728395
22 	 0.45454545454545453
0.45454545454545453
<_io.TextIOWrapper name='/home/ritam/Desktop/LeDAM/ledam/DATA/Task_1/Train_catches/case_71_catchwords.txt' mode='r' encoding='UTF-8'>
['appeal', 'compromise', 'convert', 'converted', 'cost', 'date', 'decree', 'dispute', 'form', 'grant', 'history', 'information', 'issue', 'judgment', 'order', 'parties', 'partition', 'pendency', 'representative', 'second appeal', 'settlement', 'terms']

{'form', 'judgment', 'converted', 'information', 'convert', 'date', 'second appeal', 'history', 'order', 'grant'}

{'', 'thank', 'representative', 'issue', 'parties / legal representative', 'matter', 'pendency', 'terms', 'appeal', 'i.a. no', 'c', 'learned counsel for the parties', 'kurian joseph', 'disputes', 'kurian', 'counsel', 'slp', 'dispute', 'memorandum', 'joseph', 'memorandum of compromise', 'disput

In [164]:
sent_arr=['the claim','her land','their house','additional district judge','agrarian reform legislation', '11. In 1986, "Common Cause"-a Registered Society, espousing public causes, preferred a petition under Article 32 of the Constitution of India seeking certain directions.'.lower(), 'The directions need not be reproduced and suffice it to observe that by dividing the offence into two categories - those punishable with imprisonment for a period not exceeding 7 years and those punishable with imprisonment for a period exceeding 7 years, the Court laid down periods of limitation by reference to which either the prosecution evidence shall be closed or the accused shall be released on bail.']

sent_2_arr=['payment of wage', 'pendency of proceeding', 'application of', 'relationship of employer and employee']

d_arr=[nlp(a) for a in sent_arr]

import nltk
grammar = ('''
    NP: {<DET>?<ADJ>*<NOUN>+} # NP
    PREP:{<ADP>|<PRT>} # prep
    PNP: {<NP><PREP><NP>} #PP
    K:{<NP>|<ADJ>}
    PNP2: {<K><PREP><PNP>+} #PP
    VB:{<VERB>|<ADV>}
    VNP:{<VB><NP>} # VNP
    
    ''')

chunkParser = nltk.RegexpParser(grammar)

def leaves(tree,elems):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() in elems):
        yield subtree.leaves()

def get_terms(tree,elems):
    for leaf in leaves(tree,elems):
        yield [w for w,t in leaf]


for sent in sent_2_arr:
    print(sent)
    tagged = nltk.pos_tag(nltk.word_tokenize(sent),tagset='universal')    
    print(tagged)
    tree = chunkParser.parse(tagged)
    terms=get_terms(tree,['NP','PNP','VNP'])
    for term in terms:
        w=''
        for word in term:
            w+=word+' '
        print(w)
            
        
    
        
# for d in d_arr:
#     print(d)
#     for w in d:
#         print(w, w.pos_, w.tag_)
#     print()

# for d in d_arr:
#     print(d)
#     for np in d.noun_chunks:
#         print(np)


payment of wage
[('payment', 'NOUN'), ('of', 'ADP'), ('wage', 'NOUN')]
payment of wage 
payment 
wage 
pendency of proceeding
[('pendency', 'NOUN'), ('of', 'ADP'), ('proceeding', 'VERB')]
pendency 
application of
[('application', 'NOUN'), ('of', 'ADP')]
application 
relationship of employer and employee
[('relationship', 'NOUN'), ('of', 'ADP'), ('employer', 'NOUN'), ('and', 'CONJ'), ('employee', 'NOUN')]
relationship of employer 
relationship 
employer 
employee 
